In [1]:
import numpy as np
import DnnLib
import json

prueba = np.load("fashion_mnist_train.npz")
test = np.load("fashion_mnist_test.npz")

imagenes = prueba["images"]
labelE = prueba["labels"]

imagens = test["images"]
labelP = test["labels"]

imagenE = imagenes.reshape(-1, 784)/255
imagenP = imagens.reshape(-1, 784) /255


print(imagenE.shape, imagenP.shape)


(60000, 784) (10000, 784)


In [12]:
#funcion para cargar los datos del entrenamiento de practica4
def load_datos():

    with open("mnist_entrenado.json", "r") as f:
        datos = json.load(f)

    layer1 = DnnLib.DenseLayer(784, 128, DnnLib.ActivationType.RELU)
    layer1.set_regularizer(DnnLib.RegularizerType.L2, lambda_value=0.001)
    #solo un dropout ya que solo hay una capa oculta
    dropout1 = DnnLib.Dropout(dropout_rate=0.5)
    layer2 = DnnLib.DenseLayer(128, 10, DnnLib.ActivationType.SOFTMAX)
    
    #set regularizadores de L2 siendo estos los mas compatibles
    layer2.set_regularizer(DnnLib.RegularizerType.L2, lambda_value=0.001)

    #cargar los datos del json
    layer1.weights = np.array(datos["layers"][0]["W"])
    layer1.bias = np.array(datos["layers"][0]["b"])

    layer2.weights = np.array(datos["layers"][1]["W"])
    layer2.bias = np.array(datos["layers"][1]["b"])

    #retornar las capas y el dropout
    return [layer1, droupout1, layer2]

In [3]:
def entrenar_cero():

    layer1 = DnnLib.DenseLayer(784, 128, DnnLib.ActivationType.RELU)
    layer2 = DnnLib.DenseLayer(128, 10, DnnLib.ActivationType.SOFTMAX)

    return [layer1, layer2]

In [4]:
def entrenamiento(capas, optimizers, x, y, label, epochs, batches = 128):
    n = x.shape[0]
    for e in epochs:
        r = np.random.permutation(n)
        x_shuffled = x[r]
        y_shuffled = y[r]
        labels = label[r]

        epoch_loss = 0.0
        n_batches, correcto, total = 0, 0, 0
        
        for i in range(0, n, batches):
            x_batch = x_shuffled[i:i+batches]
            y_batch = y_shuffled[i:i+batches]
            label_batch = labels[i:i+batches]
            
            out1 = capas[0].forward(x_batch)
            out2 = capas[1].forward(out1)
    
            perdida = DnnLib.cross_entropy(out2, y_batch)
            
            total_reg_loss = 0.0
            for layer in layers:
                total_reg_loss += layer.compute_regularization_loss()
                total_loss = data_loss + total_reg_loss
    
            gradiente = DnnLib.softmax_crossentropy_gradient(out2, y_batch)
            gradiente = capas[1].backward(gradiente)
            gradiente = capas[0].backward(gradiente)
    
            for layer in layers:
                if not hasattr(layer, 'training'): # Dense layer
                    optimizer.update(layer)

            epoch_loss += perdida
            n_batches += 1
            
            preds = np.argmax(out2, axis=1)
            correct += np.sum(preds == label_batch)
            total += len(label_batch)
        
        avg_loss = epoch_loss / n_batches
        acc = correct/total
        
        print(f"Epoch {e}, Avg Loss: {avg_loss:.4f}, Avg Acc: {accuracy*100:.4f}",
                f"Reg Loss: {total_reg_loss:.4f}, Total: {total_loss:.4f}")

        

In [13]:
la = labelE.shape[0]
y = np.zeros((la, 10), dtype=np.float64)
y[np.arange(la), labelE] = 1.0

optimizers = DnnLib.Adam(0.001)

capas = load_datos()
entrenamiento(capas, optimizers, imagenE, y, labelE, 10)


TypeError: set_regularizer(): incompatible function arguments. The following argument types are supported:
    1. (self: DnnLib.DenseLayer, arg0: regularization::RegularizerType, arg1: typing.SupportsFloat) -> None

Invoked with: <DnnLib.DenseLayer object at 0x7fffd108af30>, <RegularizerType.L2: 2>; kwargs: lambda_value=0.001

In [ ]:
#intento de la creación de softmax_crossentropy_gradient
def softmax(x):
    exp = np.exp(x - np.max(x))
    return exp/np.sum(exp)

def gradientes(salida, one_hot):
    x = sp.Symbol('x')
    sofi = softmax(x)
    sofid= sp.diff(sofi, x)

    cross = DnnLib.cross_entropy_gradiente(salida, one_hot)

    return sofid * cross

In [ ]:
def forward_pass_with_dropout(layers, x, training=True):
    """Forward pass through network with dropout layers"""
    activation = x
    for layer in layers:
        if hasattr(layer, 'training'): # It's a dropout layer
            layer.training = training
            activation = layer.forward(activation)
        else: # It's a dense layer
            activation = layer.forward(activation)
    return activation

In [ ]:
def backward_pass_with_dropout(layers, grad_output):
    """Backward pass through network with dropout layers"""
    grad = grad_output
    for layer in reversed(layers):
        if hasattr(layer, 'training'): # It's a dropout layer
            grad = layer.backward(grad)
        else: # It's a dense layer
            grad = layer.backward(grad)
    return grad